In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

In [11]:
os.listdir()

['.git',
 '.gitignore',
 'extract_files.ipynb',
 'README.md',
 'test',
 'train',
 'try.ipynb',
 'valid']

In [12]:
train_dir = 'train'
validation_dir = 'valid'

In [13]:
# Define the base model using pre-trained weights of VGG16
base_model = applications.VGG16(weights='imagenet', include_top=False)

In [14]:
# Freeze the base model layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

In [15]:
# Add new layers for fine-tuning
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(256, activation='relu')(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(14, activation='softmax')(top_model)

# Combine the base model and the fine-tuned layers
model = tf.keras.Model(inputs=base_model.input, outputs=top_model)

In [18]:
# Create ImageDataGenerator objects for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 260915 images belonging to 14 classes.
Found 47531 images belonging to 14 classes.


In [21]:
# Convert labels to categorical format
# train_generator.labels = to_categorical(train_generator.labels)
# validation_generator.labels = to_categorical(validation_generator.labels)

# Recompile the model with the updated output layer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
def fix_logits_and_labels_error(model, train_generator, validation_generator, epochs=10):
    """
    Fixes the "logits and labels must be broadcastable" error by adjusting the model or labels.

    Args:
        model: The Keras model to be trained.
        train_generator: The training data generator.
        validation_generator: The validation data generator.
        epochs: The number of epochs to train the model.

    Returns:
        None
    """

    # Check the number of classes predicted by the model
    n_classes = model.layers[-1].units

    # Check the shape of the labels
    if train_generator.labels.shape[-1] != n_classes:
        raise ValueError(
            f"Number of classes predicted by the model ({n_classes}) does not match the number of classes in the labels ({train_generator.labels.shape[-1]})")

    # Check if labels are one-hot encoded
    if not np.issubdtype(train_generator.labels.dtype, np.floating):
        train_generator.labels = tf.one_hot(train_generator.labels, n_classes)
        validation_generator.labels = tf.one_hot(
            validation_generator.labels, n_classes)

    # Train the model
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size
    )


# Call the fix_logits_and_labels_error function
fix_logits_and_labels_error(model, train_generator, validation_generator)

# Evaluate the model on the test data

loss, accuracy = model.evaluate(validation_generator)

print('Test loss:', loss)

print('Test accuracy:', accuracy)

ValueError: Number of classes predicted by the model (14) does not match the number of classes in the labels (260915)

In [34]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
# Define the pre-trained model and image size
base_model = VGG16(include_top=False, weights="imagenet")
img_width, img_height = 224, 224

# Define data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Define the training and validation data paths
train_data_path = "train"
validation_data_path = "valid"

# Configure data generators
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode="categorical",
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode="categorical",
)

# Define the output layer
output_layer = Dense(
    units=14,  # Number of classes
    activation="softmax",
    kernel_initializer="he_normal",
)

# Freeze the pre-trained model layers
for layer in base_model.layers:
    layer.trainable = False

# Combine the pre-trained model and the output layer
model = Model(inputs=base_model.input, outputs=output_layer(base_model.output))

# Compile the model
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator) // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(
        validation_generator) // validation_generator.batch_size,
)

# Save the model
model.save("model.h5")

Found 260915 images belonging to 14 classes.
Found 47531 images belonging to 14 classes.
Epoch 1/20


ValueError: in user code:

    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1139, in compute_loss
        return self.compiled_loss(
    F

In [40]:
from tensorflow.keras.callbacks import EarlyStopping
def create_model(img_width, img_height, num_classes):
    base_model = VGG16(include_top=False, weights="imagenet")
    input_layer = Input(shape=(img_width, img_height, 3))

    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Combine the input layer and the pre-trained model
    x = base_model(input_layer)
    x = GlobalAveragePooling2D()(x)

    # Add the output layer
    output_layer = Dense(units=num_classes, activation="softmax")(x)

    # Create and return the model
    model = Model(inputs=input_layer, outputs=output_layer)
    return model


# Define the model
model = create_model(img_width, img_height, 14)

model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

img_width, img_height = 224, 224

# Resize images in the data generator
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode="categorical",
)

# Update validation data generator with the same size


early_stopping = EarlyStopping(monitor="val_loss", patience=5)

# Train the model with early stopping
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator) // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(
        validation_generator) // validation_generator.batch_size,
    callbacks=[early_stopping],
)



Found 260915 images belonging to 14 classes.
Epoch 1/20
 87/254 [=========>....................] - ETA: 12:35 - loss: 2.5095 - accuracy: 0.1394

KeyboardInterrupt: 

In [ ]:
from collections import Counter

class_counts = Counter(train_generator.labels)
print(class_counts)

Counter({3: 36625, 4: 35761, 11: 30755, 8: 27463, 2: 19422, 6: 18156, 1: 17358, 5: 16973, 12: 16259, 0: 15605, 10: 9992, 13: 7271, 9: 5612, 7: 3663})
